In [ ]:
import torch
import torch.nn as nn
import torch.optim
import torchvision
import torchvision.transforms as transforms
import time

from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


Using: cuda


In [ ]:
## CIFAR-10 normalization vals(3 channels, RGB)
mean = (0.4914, 0.4822, 0.4465) ## one mean + one std per channel
std =  (0.2470, 0.2435, 0.2616)

transform = transforms.Compose([
    transforms.ToTensor(), ## convert PIL img to tensor, convert pixel vals to 0-1 float
    transforms.Normalize(mean, std)

])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform) ## load training split of CIFAR-10
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform) ## load the test set of 10k images

## load data in mini-batches
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 170M/170M [00:04<00:00, 41.7MB/s]


In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(3072, 512), # 3072 input features, 512 hidden neurons
    nn.ReLU(),
    nn.Linear(512, 10) # 512 hidden features, 10 output classes
)
print(model)
model = model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=3072, out_features=512, bias=True)
  (2): ReLU()
  (3): Linear(in_features=512, out_features=10, bias=True)
)


In [ ]:
# Loss, Optimizer, Training setup
loss_fn = nn.CrossEntropyLoss() ## combination of softmax and Nllloss
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
num_epochs = 10


# Training Loop

train_losses = []
train_accuracies = []
epoch_times = []

for epoch in range(num_epochs):
    start = time.time()
    model.train()

    running_loss = 0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

        # Compute accuracy
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    # Metrics for the epoch
    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    elapsed = time.time() - start

    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)
    epoch_times.append(elapsed)

    print(f"Epoch {epoch+1}/{num_epochs}  "
          f"Loss: {avg_loss:.4f}  "
          f"Accuracy: {accuracy:.4f}  "
          f"Time: {elapsed:.2f}s")

Epoch 1/10  Loss: 896.0957  Accuracy: 0.2299  Time: 13.18s
Epoch 2/10  Loss: 902.4521  Accuracy: 0.2421  Time: 12.41s
Epoch 3/10  Loss: 882.8511  Accuracy: 0.2487  Time: 12.13s
Epoch 4/10  Loss: 899.0643  Accuracy: 0.2530  Time: 11.74s
Epoch 5/10  Loss: 797.2623  Accuracy: 0.2490  Time: 11.99s
Epoch 6/10  Loss: 868.5199  Accuracy: 0.2557  Time: 11.82s
Epoch 7/10  Loss: 808.7268  Accuracy: 0.2558  Time: 11.93s
Epoch 8/10  Loss: 759.5083  Accuracy: 0.2592  Time: 11.99s
Epoch 9/10  Loss: 675.6210  Accuracy: 0.2613  Time: 11.78s
Epoch 10/10  Loss: 714.0831  Accuracy: 0.2551  Time: 11.83s


In [ ]:
import torch
from torchvision import datasets, transforms

# Load raw CIFAR-10 (no normalization)
transform = transforms.ToTensor()
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

loader = torch.utils.data.DataLoader(dataset, batch_size=50000, shuffle=False)

images, _ = next(iter(loader))  # all 50k images

# Compute per-channel mean and std
mean = images.mean(dim=[0,2,3])
std = images.std(dim=[0,2,3])

print("Mean:", mean)
print("Std:", std)

100%|██████████| 170M/170M [00:03<00:00, 44.5MB/s]


Mean: tensor([0.4914, 0.4822, 0.4465])
Std: tensor([0.2470, 0.2435, 0.2616])
